In [1]:
import json
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.probability import FreqDist
from nltk import trigrams
from nltk import bigrams

Read Data

In [5]:
my_data=[]
with open("signal-news1.jsonl","r")as f:
    for lines in f:
        my_data.append(json.loads(lines))

# PART A

Change all characters to lower case

In [6]:
for i in my_data:
    my_data[my_data.index(i)]=i['content'].lower()

Each pattern represent a task in part A

In [7]:
pattern1="(http://[A-Za-z0-9]+(\.[A-Za-z0-9]*)*(/[A-Za-z0-9]*)*)"   #match url
pattern2="\W"   #match non-alphabetic
pattern3=" [A-Za-z] " # 3 and 4 match single character
pattern4="^[A-Za-z] "
pattern5="([^A-Za-z]+[0-9]+)|([0-9]*[^A-Za-z]+)"   #match pure digits

In [8]:
for i in my_data:
    a=re.sub(pattern1,'',i)
    b=re.sub(pattern2,' ',a) #keep white space
    c=re.sub(pattern3,' ',b)
    d=re.sub(pattern4,'',c)
    e=re.sub(pattern5,' ',d)
    my_data[my_data.index(i)]=nltk.word_tokenize(e)

Lemmitize tokens

In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
'''
from nltk.corpus import wordnet
def get_word_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

for news in my_data:
    news_pos=nltk.pos_tag(news)
    for i in range(len(news)):
        word_pos=get_word_pos(news_pos[i][1])
        if word_pos!='':
            news[i]=lemmatizer.lemmatize(news[i],pos=word_pos)
'''

In [7]:
for news in my_data:
    for i in range(len(news)):
        news[i]=lemmatizer.lemmatize(news[i])

# PART B

To build a corpus which consists of tokens

In [8]:
corpus=[]
for i in my_data:
    corpus.extend(i)

Compute the size of tokens and vocabulary

In [9]:
N=len(corpus)
vocab=set(corpus)
V=len(vocab)

List the top 25 trigrams

In [10]:
tri=trigrams(corpus)
f=FreqDist(tri)
top_tri=f.most_common(25)
top_tri=[i[0] for i in top_tri]
print(top_tri)

[('one', 'of', 'the'), ('on', 'share', 'of'), ('day', 'moving', 'average'), ('on', 'the', 'stock'), ('a', 'well', 'a'), ('in', 'research', 'report'), ('in', 'research', 'note'), ('the', 'year', 'old'), ('the', 'united', 'state'), ('for', 'the', 'quarter'), ('average', 'price', 'of'), ('research', 'report', 'on'), ('research', 'note', 'on'), ('the', 'end', 'of'), ('share', 'of', 'the'), ('in', 'report', 'on'), ('earnings', 'per', 'share'), ('buy', 'rating', 'to'), ('cell', 'phone', 'plan'), ('phone', 'plan', 'detail'), ('according', 'to', 'the'), ('of', 'the', 'company'), ('appeared', 'first', 'on'), ('moving', 'average', 'price'), ('price', 'target', 'on')]


Read the file of positive and negative words

In [13]:
positive_words=[]
negative_words=[]


with open("opinion-lexicon-English/positive-words.txt",'r') as f:
    
    positive_words=f.read().splitlines()
    
with open("opinion-lexicon-English/negative-words.txt",'r') as f:
    
    negative_words=f.read().splitlines()



Use binary search to find words in positive or negative words to make it faster

In [14]:
def binary_search(lis, word):
        left = 0
        right = len(lis) - 1
        while left <= right:   
            mid = (left + right)//2   
            if word < lis[mid]:  
                right = mid - 1   
            elif word > lis[mid]:   
                left = mid + 1    
            else:
                return mid  
        return -1


Compute the number of positive and negative word counts in the corpus.

In [15]:
pos=0
neg=0

for i in corpus:
    w1=binary_search(positive_words,i)
    if w1!=-1:
        pos+=1
    w2=binary_search(negative_words,i)
    if w2!=-1:
        neg+=1
print(pos,neg)

176961 132695


Compute the number of news stories with more positive than negative words, as well as the number of news stories with more negative than positive words

In [15]:
pos_news=0
neg_news=0

for news in my_data:
    pos=0
    neg=0
    
    for i in news:
        w1=binary_search(positive_words,i)
        if w1!=-1:
            pos+=1
        w2=binary_search(negative_words,i)
        if w2!=-1:
            neg+=1
    if pos>neg:
        pos_news+=1
    elif pos<neg:
        neg_news+=1

print(pos_news,neg_news)

10584 6825


# PART C

In [16]:
cor1=[]
for i in my_data[:16000]:
    cor1.extend(i)
vocab1=set(cor1)

Laplas Smoothing Trigrams. 
To add 1 to each trigram and add size of bigrams set to each bigram

In [17]:
tri1=trigrams(cor1)
ft1=FreqDist(tri1)
count_t=[i for i in ft1.values()]
count_t=list(map(lambda x :x+1,count_t))
trigram=[i for i in ft1.keys()]
tri1=dict(zip(trigram,count_t))

In [18]:
bi1=[i for i in bigrams(cor1)]
fb1=FreqDist(bi1)
V2=len(set(bi1))
count_b=[i for i in fb1.values()]
count_b=list(map(lambda x :x+V2,count_b))
bigram=[i for i in fb1.keys()]
bi1=dict(zip(bigram,count_b))

Define a function that could compute the probability of P(w3|w1,w2) and change it to log value

In [16]:
import math
def prob(w1,w2,w3):
    if (w1,w2,w3) in tri1 and (w1,w2) in bi1:
        p=tri1[(w1,w2,w3)]/bi1[(w1,w2)]
        log_p=math.log(p)
        return log_p

Generate the sentence starting with 'is this'

In [20]:
w1='is'
w2='this'
sen=['is','this']
for i in range(8):
    score=[]
    words=[]
    for v in vocab1:
        log_p=prob(w1,w2,v)
        if log_p !=None:
            score.append(round(log_p,5))
            words.append(v)
    w3=words[score.index(max(score))]
    w1=w2
    w2=w3
    print(w3)
    sen.append(w3)

sentence=' '.join(sen)
print(sentence)

the
company
ha
market
capitalization
of
billion
and
is this the company ha market capitalization of billion and


Read the remaining rows and build a token corpus

In [19]:
cor2=[]
for i in my_data[16000:]:
    cor2.extend(i)
vocab2=set(cor2)

Laplace smooting the remaining rows

In [20]:
tri2=trigrams(cor2)
ft2=FreqDist(tri2) # the dict to store (trigram-number of trigram) pairs

count_t2=[i for i in ft2.values()]  
count_t2=list(map(lambda x :x+1,count_t2))
trigram2=[i for i in ft2.keys()]
tri2=dict(zip(trigram2,count_t2))

bi2=[i for i in bigrams(cor2)]
fb2=FreqDist(bi2)

V22=len(set(bi2))

count_b2=[i for i in fb2.values()]
count_b2=list(map(lambda x :x+V22,count_b2))
bigram2=[i for i in fb2.keys()]
bi2=dict(zip(bigram2,count_b2))

Define another probability function with the next corpus to compute the probability and change it to log

In [21]:
def prob2(w1,w2,w3):
    p=tri[(w1,w2,w3)]/bi2[(w1,w2)]
    log_p=math.log(p)
    return log_p

Compute the perplexity of remaing corpus

In [23]:
pp=[]
for news in my_data[16000:18000]:
    t=trigrams(news)
    b=bigrams(news)
    w1=news[0]
    w2=news[1]
    total_log=[]
    for i in news[2:]:
        log_p=prob2(w1,w2,i)
        if log_p!=None:
            total_log.append(log_p)
        w1=w2
        w2=i
    sum_t=sum(total_log)
    perplexity=pow(1/abs(sum_t),1/len(news))
    pp.append(perplexity)

-5417.888050368112
-2157.632805442196
-1659.5830838465345
-2225.285531589915
-693.4574602940743
-6503.582672285937
-4483.723888043148
-7486.816603154829
-7466.731703148335
-5107.715682919934
-2257.9053431074794
-2643.84531192084
-2911.341643160415
-2861.0050605694028
-440.25188721291136
-4034.3561758693213
-1859.0738035183706
-2867.592100428679
-2092.3022967302504
-4202.924743453175
-345.51720090327666
-7576.284519023398
-231.23166594166773
-1015.6017770351505
-895.543781472596
-4027.8241804937697
-240.64627296325386
-5091.202424908502
-4466.873502988699
-23029.534449783467
-1018.062672782601
-9303.502242951503
-3315.1208104450366
-3084.69920922883
-1573.8828039107889
-3218.342354105434
-2623.50975031641
-1964.7346539500895
-11148.566429708535
-2047.5013293899235
-313.99536913983957
-4309.657463170777
-960.2596869626177
-3591.3331400479406
-4113.464804612985
-4362.705607529609
-329.34482572148556
-3043.9639721533576
-2602.698017573317
-1329.9011145088982
-3942.8322942906752
-3023.97465

-8349.652903408354
-2427.16104570162
-2705.7266617391374
-1267.6990635572033
-2967.766035252152
-8070.672416609323
-3698.1552113432917
-94817.51388300509
-2860.498679621725
-6190.875064261588
-4354.5872803189695
-14642.235660227232
-4768.436984994729
-1730.438059470153
-3370.181766120409
-1813.1014395296168
-2916.4154947860247
-1607.083224909427
-2344.5112429133906
-2314.6840377336775
-9937.008637651024
-2306.9583069694963
-3868.141266235469
-3729.045775221463
-2769.926163772136
-15488.171132618221
-5881.114196751899
-4145.9976150420625
-10877.022022052319
-276.19801045818417
-3549.8387272923574
-8991.283935410453
-7373.152905843146
-347.83722637010754
-332.91931839746786
-966.7780140741032
-1224.0793322029042
-618.9774154931744
-563.7210284821898
-7921.498217309045
-623.5102459805099
-2373.459632560755
-3575.819502558216
-3963.5277165287002
-8406.67120897855
-1854.799514303977
-4345.819146851962
-3221.6055579680146
-2828.4249893084925
-1855.3460316357407
-2864.1305389861272
-401.73584

-7180.998553940531
-4668.01238046708
-4703.603163482929
-1246.297641889503
-4861.718245112986
-319.4789249281293
-650.0150214452905
-1005.9455221961207
-2731.3462835089576
-4300.003934707996
-4362.262855219196
-5718.607753668067
-4375.197681955276
-2337.2295064998225
-3643.585322357557
-7143.082614428849
-320.89324781099094
-3414.889458909176
-499.292051490423
-263.85470136760046
-204.97530417450267
-1262.1297189204454
-4338.353764201154
-1255.9560687880287
-3175.265621741048
-564.5878032834682
-1848.4041615063134
-8138.035298093046
-775.9445823668541
-1038.350756396703
-6685.987615330502
-1246.0574860051963
-1183.45355636021
-672.3301277444479
-317.61815191407084
-4443.050215558701
-588.2490382613765
-2637.7412796018316
-3558.0588107039957
-2170.2622791956733
-2325.609597453026
-3046.311976126874
-2578.705348199242
-5881.304663093214
-595.0392727778939
-628.0758226935865
-3329.5169739966404
-1421.9998900269788
-6331.008252096709
-1644.1869790878416
-3768.8327517464636
-1930.0125544720

In [32]:
perp=sum(pp)/len(pp)
print(perp)

[0.9816361188036217, 0.9604883391960776, 0.9493271527325312, 0.9611072814948799, 0.9023723228202, 0.9846814771207408, 0.9785463607178092, 0.9860668230420786, 0.9861727151826679, 0.9819658918875347, 0.9614306959771104, 0.9722481359834809, 0.9686340418795771, 0.9689151765930739, 0.8820999709242596, 0.9763037519981005, 0.9547042093616077, 0.9684247150848558, 0.9589313156866948, 0.9832858408959156, 0.8277698383660945, 0.9862343662017028, 0.7873028140317946, 0.9257088654589984, 0.9181500676814239, 0.9768485646288066, 0.7791704890907256, 0.981782757884034, 0.9780351030743532, 0.9950518945738577, 0.9256421466330648, 0.9887665320697001, 0.9782394109918767, 0.9756942537936736, 0.9495935866727382, 0.9770596706943696, 0.9667326360108244, 0.9576088999636159, 0.9902872614860645, 0.9657634497366122, 0.8243251741240804, 0.9779670437931879, 0.9220977776743988, 0.974013891809148, 0.9771256575020885, 0.9783803300405772, 0.8236162862053577, 0.9696652426229585, 0.9655567695961426, 0.9410928447875223, 0.98